<a href="https://colab.research.google.com/github/AndreSlavescu/Intermediate-Gauss-Seidel-Decoding/blob/main/Intermediate_Gauss_Seidel_Decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!nvidia-smi

Tue Oct 29 18:25:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=1d838fc77c03fe9648f6c4179f8e67fe6c576a1e9ac94a2c3cf620efd8338d30
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


# Idea and Motivation



## Idea

The Gauss-Seidel Iteration Method, while parallel in computing the components of the vector $x$ in $Ax = b$, traditionally suffers from a sequential bottleneck when performing multiple iterations $n > 1$. To enhance parallelism, we introduce a "jump iteration" approach:

- **Alternating Updates:** Compute even indices of $x$ on even iterations and odd indices on odd iterations.
- **Parallelism:** This allows for a two-fold increase in parallelism, as updates within each set (even or odd) can be performed simultaneously.
- **Double Buffering:** By utilizing two buffers, we can manage read and write operations without data hazards.

## Motivation

The main motivator behind this exploration has been the parallel decoding infrastructure seen in [lookahead decoding](https://lmsys.org/blog/2023-11-21-lookahead-decoding/).

# Method

### Mathematical Formulation

The update rule for the Alternating Update Method is defined as:

- **For iteration $k$:**

  - **If $k$ is even**, update $x_i^{(k+1)}$ for even $i$:

    $x_i^{(k+1)} = \frac{1}{A_{ii}} \left( b_i - \sum_{j \neq i} A_{ij} x_j^{(k)} \right)$

  - **If $k$ is odd**, update $x_i^{(k+1)}$ for odd $i$:

    $x_i^{(k+1)} = \frac{1}{A_{ii}} \left( b_i - \sum_{j \neq i} A_{ij} x_j^{(k)} \right)$

- **Non-updated indices retain their previous values:**

  $x_i^{(k+1)} = x_i^{(k)}$, $\quad$ if $i$ is not updated at iteration $k$

This approach leverages double buffering to separate read and write operations, enabling parallel updates within each set of indices.


In [2]:
import numpy as np
import numpy.linalg as la
import random

import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from pycuda.driver import Event

# typing
from typing import List, Tuple

# Alternating Gauss Seidel Method
alternating_kernel_code = '''
__global__ void alternating_gauss_seidel(
    float *A, float *b, float *x_1, float *x_2,
    int size, int iterations)
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index >= size) return;

    float *x_read, *x_write;

    // Initialize both buffers
    x_1[index] = 0.0f;
    x_2[index] = 0.0f;
    __syncthreads();

    for (int iter = 0; iter < iterations; ++iter) {
        // Determine the read and write buffers
        if (iter % 2 == 0) {
            x_read = x_1;
            x_write = x_2;
        } else {
            x_read = x_2;
            x_write = x_1;
        }

        // Determine if index should be updated in this iteration
        bool update_index = (iter % 2 == index % 2);

        if (update_index) {
            float sum = 0.0f;
            for (int j = 0; j < size; ++j) {
                if (j != index) {
                    sum += A[index * size + j] * x_read[j];
                }
            }
            x_write[index] = (b[index] - sum) / A[index * size + index];
        } else {
            x_write[index] = x_read[index];
        }
        __syncthreads();
    }

    // Copy final result to x_1 if needed
    if (iterations % 2 != 0) {
        x_1[index] = x_2[index];
    }
}
'''

# Red-Black Gauss Seidel Method
red_black_kernel_code = '''
__global__ void red_black_gauss_seidel(
    float *A, float *b, float *x,
    int size, int iterations)
{
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index >= size) return;

    x[index] = 0.0f;
    __syncthreads();

    for (int iter = 0; iter < iterations; ++iter) {
        // Red update: even indices
        bool is_red = (index % 2 == 0);
        bool is_red_phase = (iter % 2 == 0);

        if (is_red_phase && is_red) {
            float sum = 0.0f;
            for (int j = 0; j < size; ++j) {
                if (j != index) {
                    sum += A[index * size + j] * x[j];
                }
            }
            x[index] = (b[index] - sum) / A[index * size + index];
        }

        if (!is_red_phase && !is_red) {
            float sum = 0.0f;
            for (int j = 0; j < size; ++j) {
                if (j != index) {
                    sum += A[index * size + j] * x[j];
                }
            }
            x[index] = (b[index] - sum) / A[index * size + index];
        }
        __syncthreads();
    }
}
'''

# Compile the kernel code
mod_alt = SourceModule(alternating_kernel_code)
alternating_gauss_seidel = mod_alt.get_function("alternating_gauss_seidel")

mod_rb = SourceModule(red_black_kernel_code)
red_black_gauss_seidel = mod_rb.get_function("red_black_gauss_seidel")

def run_alternating_gauss_seidel_gpu(
    A: np.ndarray,
    b: np.ndarray,
    x0: np.ndarray,
    size: int,
    iterations: int
) -> np.ndarray:
    A_gpu = cuda.mem_alloc(A.nbytes)
    b_gpu = cuda.mem_alloc(b.nbytes)
    x_gpu_1 = cuda.mem_alloc(x0.nbytes)
    x_gpu_2 = cuda.mem_alloc(x0.nbytes)

    cuda.memcpy_htod(A_gpu, A)
    cuda.memcpy_htod(b_gpu, b)
    cuda.memcpy_htod(x_gpu_1, x0)

    block_size = 256
    grid_size = (size + block_size - 1) // block_size

    start = cuda.Event()
    end = cuda.Event()
    start.record()

    alternating_gauss_seidel(
        A_gpu, b_gpu, x_gpu_1, x_gpu_2,
        np.int32(size), np.int32(iterations),
        block=(block_size, 1, 1), grid=(grid_size, 1)
    )

    end.record()
    end.synchronize()
    elapsed_time = start.time_till(end)

    x_result = np.empty_like(b)
    cuda.memcpy_dtoh(x_result, x_gpu_1)
    print(f"Alternating Gauss-Seidel execution time: {elapsed_time:.5f} milliseconds")

    A_gpu.free()
    b_gpu.free()
    x_gpu_1.free()
    x_gpu_2.free()

    return x_result

def run_red_black_gauss_seidel_gpu(
    A: np.ndarray,
    b: np.ndarray,
    x0: np.ndarray,
    size: int,
    iterations: int
) -> np.ndarray:
    A_gpu = cuda.mem_alloc(A.nbytes)
    b_gpu = cuda.mem_alloc(b.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)

    cuda.memcpy_htod(A_gpu, A)
    cuda.memcpy_htod(b_gpu, b)
    cuda.memcpy_htod(x_gpu, x0)

    block_size = 256
    grid_size = (size + block_size - 1) // block_size

    start = cuda.Event()
    end = cuda.Event()
    start.record()

    red_black_gauss_seidel(
        A_gpu, b_gpu, x_gpu,
        np.int32(size), np.int32(iterations),
        block=(block_size, 1, 1), grid=(grid_size, 1)
    )

    end.record()
    end.synchronize()
    elapsed_time = start.time_till(end)

    x_result = np.empty_like(b)
    cuda.memcpy_dtoh(x_result, x_gpu)
    print(f"Red-Black Gauss-Seidel Method execution time: {elapsed_time:.5f} milliseconds")

    A_gpu.free()
    b_gpu.free()
    x_gpu.free()

    return x_result

def generate_test_equation(size: int) -> Tuple[np.ndarray, np.ndarray]:
    A = np.random.rand(size, size).astype(np.float32)

    # diagonally dominant cases
    np.fill_diagonal(A, np.sum(np.abs(A), axis=1) + 1)
    b = np.random.rand(size).astype(np.float32)
    return A, b


def test_methods(
    size: int,
    iterations: int = 100
):
    A, b = generate_test_equation(size)
    x0 = np.zeros_like(b).astype(np.float32)

    print(f"Testing with system size: {size}, iterations: {iterations}\n")

    x_alt = run_alternating_gauss_seidel_gpu(A, b, x0, size, iterations)
    x_rb = run_red_black_gauss_seidel_gpu(A, b, x0, size, iterations)
    x_exact = la.solve(A, b)

    error_alt = np.linalg.norm(x_alt - x_exact)
    error_rb = np.linalg.norm(x_rb - x_exact)

    print(f"Alternating Method Error: {error_alt:.5e}")
    print(f"Red-Black Method Error: {error_rb:.5e}")

    if error_alt < error_rb:
        print("\nAlternating Update Method converged closer to the exact solution.")
    else:
        print("\nRed-Black Gauss-Seidel Method converged closer to the exact solution.")


if __name__ == "__main__":
    num_tests = 10
    test_size = 1000
    num_iterations = 100

    for i in range(num_tests):
      print(f"\n\nTest {i + 1}/{num_tests}:\n\n")
      test_methods(test_size, num_iterations)



Test 1/10:


Testing with system size: 1000, iterations: 100

Alternating Gauss-Seidel execution time: 21.63472 milliseconds
Red-Black Gauss-Seidel Method execution time: 21.31165 milliseconds
Alternating Method Error: 6.60348e-09
Red-Black Method Error: 6.62116e-09

Alternating Update Method converged closer to the exact solution.


Test 2/10:


Testing with system size: 1000, iterations: 100

Alternating Gauss-Seidel execution time: 21.28486 milliseconds
Red-Black Gauss-Seidel Method execution time: 21.33194 milliseconds
Alternating Method Error: 6.27281e-09
Red-Black Method Error: 6.28376e-09

Alternating Update Method converged closer to the exact solution.


Test 3/10:


Testing with system size: 1000, iterations: 100

Alternating Gauss-Seidel execution time: 21.28144 milliseconds
Red-Black Gauss-Seidel Method execution time: 21.34630 milliseconds
Alternating Method Error: 7.00798e-09
Red-Black Method Error: 7.01324e-09

Alternating Update Method converged closer to the exact so